In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import pairwise_distances
from heapq import nsmallest

In [2]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

In [3]:
embed = hub.load(module_url)

In [4]:
articles = pd.read_csv('../dataset/data_articles_test.csv', delimiter=',', keep_default_na=False)

tables = pd.read_csv('../dataset/distinct_tables_allsignals.csv', delimiter=',', keep_default_na=False)
tables = tables.drop(tables[tables.table_page_title == ""].index)

## creating the articles embedings

In [5]:
articles_title = []
articles_id = []

for i, row in articles.iterrows():
    
    articles_id.append(row['page_id'])
    articles_title.append(row['page_title']+" "+row['summary']+" "+row['keywords'])

In [6]:
embedding_articles = embed(articles_title)

In [7]:
article_dense_vector = []

for current_embedding in embedding_articles:
    
    article_dense_vector.append(current_embedding.numpy())

## creating the tables embeddings

In [8]:
tables_title = []
tables_ids = []

for i, row in tqdm(tables.iterrows()):
    
    tables_ids.append(row['table_id'])
    tables_title.append(row['table_page_title']+" "+row['table_page_summary']+" "+row['table_page_keywords'])

85900it [00:12, 6724.10it/s]


In [9]:
embedding_tables = embed(tables_title)

In [10]:
tables_dense_vector = []

for current_embedding in embedding_tables:
    
    tables_dense_vector.append(current_embedding.numpy())

## methods for getting the ranked tables and save the accuracy

In [11]:
def get_accuracy(id_ranked_tables, id_query_goal):

    accuracy = 0

    for id_table in id_ranked_tables:
    
        if id_table[0] == id_query_goal:
    
            accuracy = 1
            
            break;

    return accuracy

In [12]:
def save_accuracy(k,accuracy):
    
    if k == 1:
            
        average_top1.append(accuracy)
        
    if k == 5:
            
        average_top5.append(accuracy)
        
    if k == 10:
            
        average_top10.append(accuracy)
        
    if k == 20:
            
        average_top20.append(accuracy)

## computing the cosine similarity

In [13]:
average_top1 = []
average_top5 = []
average_top10 = []
average_top20 = []

top_k = [1,5,10,20]

for i in tqdm(range(len(article_dense_vector))):
    
    idQueryGoal = articles_id[i]
    
    distance_vector = pairwise_distances(article_dense_vector[i].reshape(1,512), tables_dense_vector, metric='cosine')
    
    #creating the dataframe
    all_tables_score = []
    
    for j in range(len(tables_ids)):
        
        table_id = tables_ids[j]
        table_score = distance_vector[0][j]
        
        new_row = {"table_id": table_id,"table_score": table_score}
        
        all_tables_score.append(new_row)
        
    df_all_tables_scores = pd.DataFrame(all_tables_score)
    df_tables_sorting = df_all_tables_scores.sort_values('table_score')
    
    #compute the accuracy
    for accuracyK in top_k:
        
        selected_top = df_tables_sorting.head(accuracyK)
        min_score = selected_top['table_score'].max()
        draw_tables_socres = df_tables_sorting[df_tables_sorting['table_score'] <= min_score]
        final_ranked_tables = draw_tables_socres.iloc[:,0:1].values
        
        accuracy_value = get_accuracy(final_ranked_tables,idQueryGoal)
        
        #save the accuracy on the list
        save_accuracy(accuracyK,accuracy_value)

100%|██████████| 1026/1026 [06:50<00:00,  2.47it/s]


In [14]:
print(str(round(np.mean(average_top1),4)))
print(str(round(np.mean(average_top5),4)))
print(str(round(np.mean(average_top10),4)))
print(str(round(np.mean(average_top20),4)))

0.1647
0.2729
0.3148
0.3694
